In [5]:
# %env MISTRAL_API_KEY=

In [14]:
import os
MISTRAL_API_KEY = os.environ["MISTRAL_API_KEY"]

In [ ]:
from autogen import AssistantAgent, UserProxyAgent

# build the llm_config, I think that I can add multiple configs for multiple agents
llm_config = { "config_list":
              [{ "model": "open-mistral-7b", "api_key": MISTRAL_API_KEY, "api_type": "mistral", "max_tokens": 1000 }]
              }

# build an agent
general_assistant = AssistantAgent(name="general_assistant", llm_config=llm_config)
user_proxy = UserProxyAgent("user_proxy", code_execution_config=False)


In [ ]:
user_proxy.initiate_chat(
    general_assistant, 
    message="explain to me the process of driving a car."
)

user_proxy (to general_assistant):

explain to me the process of driving a car.

--------------------------------------------------------------------------------
general_assistant (to user_proxy):

To explain the process of driving a car, I will first describe the steps in plain language, and then provide a simple Python script that simulates a basic car driving simulation.

Steps to drive a car:
1. Sit in the driver's seat with the seat belt fastened.
2. Check the mirrors (rear-view, side-view, and interior) to ensure your surroundings are clear.
3. Put the car in gear (usually first gear), release the parking brake, and apply the foot brake gently to keep the car stationary.
4. Gradually release the foot brake while simultaneously pressing the accelerator to move the car forward.
5. Steer the wheel to control the direction of the car.
6. If necessary, use the clutch, gear shift, and brakes to change speed or direction.
7. Be aware of traffic rules, signs, and other road users to ensu

KeyboardInterrupt: 

## Two Agents coding example
 two-agent chat with an AssistantAgent (primarily a coding agent) to generate code to count the number of prime numbers between 1 and 10,000 and then it will be executed.

In [43]:
from pathlib import Path
from autogen.coding import LocalCommandLineCodeExecutor

# Setting up the code executor
workdir = Path("coding")
workdir.mkdir(exist_ok=True)
code_executor = LocalCommandLineCodeExecutor(work_dir=workdir)

# build the llm_config, I think that I can add multiple configs for multiple agents
llm_config = { "config_list":
              [{ "model": "open-mixtral-8x22b", "api_key": MISTRAL_API_KEY, "api_type": "mistral" }]
              }



### Setup the Agents

In [44]:
from autogen import AssistantAgent, UserProxyAgent

# The UserProxyAgent will execute the code that the AssistantAgent provides
user_proxy_agent = UserProxyAgent(
    name="User",
    human_input_mode="NEVER",
    code_execution_config={"executor": code_executor},
    is_termination_msg=lambda msg: "FINISH" in msg.get("content", ""),
)

system_message = """You are a helpful AI assistant who writes code and the user executes it.
Solve tasks using your coding and language skills.
In the following cases, suggest python code (in a python coding block) for the user to execute.
Solve the task step by step if you need to. If a plan is not provided, explain your plan first. Be clear which step uses code, and which step uses your language skill.
When using code, you must indicate the script type in the code block. The user cannot provide any other feedback or perform any other action beyond executing the code you suggest. The user can't modify your code. So do not suggest incomplete code which requires users to modify. Don't use a code block if it's not intended to be executed by the user.
Don't include multiple code blocks in one response. Do not ask users to copy and paste the result. Instead, use 'print' function for the output when relevant. Check the execution result returned by the user.
If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
When you find an answer, verify the answer carefully. Include verifiable evidence in your response if possible.
IMPORTANT: Wait for the user to execute your code and then you can reply with the word "FINISH". DO NOT OUTPUT "FINISH" after your code block."""


# The AssistantAgent, using Mistral AI's model, will take the coding request and return code
assistant_agent = AssistantAgent(
    name="Mistral Assistant",
    system_message=system_message,
    llm_config=llm_config,
)


### Start the chat, with the UserProxyAgent asking the AssistantAgent the message

In [45]:
chat_result  = user_proxy_agent.initiate_chat(recipient=assistant_agent,
                                message="Provide code to count the number of prime numbers from 1 to 10000.")


User (to Mistral Assistant):

Provide code to count the number of prime numbers from 1 to 10000.

--------------------------------------------------------------------------------
Mistral Assistant (to User):

Here is a Python script that defines a function to check if a number is prime, then iterates through numbers from 1 to 10000 to find and count prime numbers.

```python
def is_prime(n):
    if n <= 1:
        return False
    if n <= 3:
        return True
    if n % 2 == 0 or n % 3 == 0:
        return False
    i = 5
    while i * i <= n:
        if n % i == 0 or n % (i + 2) == 0:
            return False
        i += 6
    return True

count = 0
for num in range(1, 10001):
    if is_prime(num):
        count += 1

print(count)
```

This script first defines a helper function `is_prime(n)` that checks if a number `n` is prime. It then iterates over numbers from 1 to 10000 and if a number is prime, it increments the `count`. Finally, it prints the total count of prime numbers.

-

In [46]:
chat_result

ChatResult(chat_id=None, chat_history=[{'content': 'Provide code to count the number of prime numbers from 1 to 10000.', 'role': 'assistant', 'name': 'User'}, {'content': 'Here is a Python script that defines a function to check if a number is prime, then iterates through numbers from 1 to 10000 to find and count prime numbers.\n\n```python\ndef is_prime(n):\n    if n <= 1:\n        return False\n    if n <= 3:\n        return True\n    if n % 2 == 0 or n % 3 == 0:\n        return False\n    i = 5\n    while i * i <= n:\n        if n % i == 0 or n % (i + 2) == 0:\n            return False\n        i += 6\n    return True\n\ncount = 0\nfor num in range(1, 10001):\n    if is_prime(num):\n        count += 1\n\nprint(count)\n```\n\nThis script first defines a helper function `is_prime(n)` that checks if a number `n` is prime. It then iterates over numbers from 1 to 10000 and if a number is prime, it increments the `count`. Finally, it prints the total count of prime numbers.', 'role': 'use

## Conversable agent

In [6]:
import os
from autogen import ConversableAgent


# build the llm_config, I think that I can add multiple configs for multiple agents
llm_config = { "config_list":
              [{ "model": "open-mistral-7b", "api_key":os.environ["MISTRAL_API_KEY"] , "api_type": "mistral" }]
              }
agent = ConversableAgent(name="chatbot", human_input_mode="never", llm_config=llm_config, code_execution_config=False, function_map=None)

flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.


In [11]:
reply = agent.generate_reply(messages=[{"content": "explain in detail the process of writing python code and turn them into packages.", "role": "user"}])
print(reply["content"])


>>>>>>>> USING AUTO REPLY...
Writing Python code and turning it into a package involves several steps. Here's a detailed process:

1. **Planning and Designing**: Before you start writing code, it's essential to plan and design your package. This includes deciding on the package's functionality, its architecture, and how different components will interact.

2. **Setting Up the Project**: Create a new directory for your project. Inside this directory, create a file named `setup.py`. This file is crucial for packaging your Python project. It contains metadata about your project, such as its name, version, author, description, and dependencies.

3. **Writing Python Code**: Start writing your Python code. Organize your code into logical modules and submodules. A module is a file containing Python definitions and statements. A submodule is a module inside another module.

4. **Testing**: Write tests for your code to ensure it works as expected. Python has built-in testing frameworks like `u

Let's say we want to agents to talk to each other, for example we want to first agent to act like an interviewer and a candidate. Then they bounce the conversation between them selves. 

In [10]:
# %env MISTRAL_API_KEY=

In [7]:
# create an agent called interviewer
interviewer_agent = ConversableAgent("interviewer",
                system_message="You are an AI who act like interviewer for a machine learning engineer position, keep your response short,  keep the conversation going",
                human_input_mode="NEVER",
                llm_config={  "config_list"  : [{"model": "open-mistral-7b", "api_key":os.environ["MISTRAL_API_KEY"] , "api_type": "mistral", "temperature": 0.7}] } )

In [8]:
# create an agent called candidate
candidate_agent = ConversableAgent("candidate",
                system_message="You are an AI act like a candidate for a machine learning engineer position, keep your response short,  keep the conversation going",
                human_input_mode="NEVER",
                llm_config={  "config_list"  : [{"model": "open-mistral-7b", "api_key":os.environ["MISTRAL_API_KEY"] , "api_type": "mistral", "temperature": 0.7 }] } )

In [9]:
result = interviewer_agent.initiate_chat(candidate_agent, message="Hello candidate!", max_turns=4)
print(result.get("content", ""))

interviewer (to candidate):

Hello candidate!

--------------------------------------------------------------------------------
candidate (to interviewer):

Hello! I'm an AI, but I've been programmed to mimic the skills and behaviors of a machine learning engineer. I have a strong background in programming languages such as Python and R, and I'm proficient in using libraries such as TensorFlow, PyTorch, and scikit-learn. I also have experience with data preprocessing, model training, and deployment. I'm excited to discuss how I can contribute to your team.

--------------------------------------------------------------------------------
interviewer (to candidate):

Greetings! It's a pleasure to meet you. Can you tell me about a recent machine learning project you worked on, including the problem you were trying to solve, the data you used, and the techniques you employed?

--------------------------------------------------------------------------------
candidate (to interviewer):

Hell

SDKError: API error occurred: Status 429
{"message":"Requests rate limit exceeded"}

### Terminate conversation between agents
1- Specify parameters in initiate_chat: When initiating a chat, you can define parameters that determine when the conversation should end. for example using the max_turns parameter to limit the number of turns

2- Configure an agent to trigger termination: When defining individual agents, you can specify parameters that allow agents to terminate of a conversation based on particular (configurable) conditions.

In [12]:
# example using max_consecutive_auto_reply
llm_config = {  "config_list"  : [{"model": "open-mistral-7b", "api_key":os.environ["MISTRAL_API_KEY"] , "api_type": "mistral", "temperature": 0.7 }] }

ash_agent = ConversableAgent(name="Ash", system_message="you are a comedian in a show between 2 comedians", max_consecutive_auto_reply=1, llm_config=llm_config, human_input_mode="NEVER")
john_agent = ConversableAgent(name="John", system_message="you are a comedian in a show between 2 comedians", max_consecutive_auto_reply=1, llm_config=llm_config, human_input_mode="NEVER")

result = ash_agent.initiate_chat(john_agent, message={"content": "hi, could you tell me a joke? " })

Ash (to John):

hi, could you tell me a joke? 

--------------------------------------------------------------------------------
John (to Ash):

Of course, I'd be happy to share a joke with you! Here it goes:

Why don't scientists trust atoms?

Because they make up everything!

And now, let me pass the mic to my fellow comedian over here. They've got a great set ready for you!

--------------------------------------------------------------------------------
Ash (to John):

Hello there! I hope you enjoyed that joke. Here's another one for you:

Why was the math book sad?

Because it had too many problems.

And now, it's time for my fellow comedian to take the stage. They've got some hilarious material prepared for you, so be sure to give them a warm welcome!

--------------------------------------------------------------------------------


In [13]:
# example using is_termination_msg
llm_config = {  "config_list"  : [{"model": "open-mistral-7b", "api_key":os.environ["MISTRAL_API_KEY"] , "api_type": "mistral", "temperature": 0.7 }] }

ash_agent = ConversableAgent(name="Ash", system_message="you are a comedian in a show between 2 comedians, say the word TERMINATE when you finish", llm_config=llm_config, human_input_mode="NEVER",
                              is_termination_msg= lambda msg: "TERMINATE" in msg["content"] )
john_agent = ConversableAgent(name="John", system_message="you are a comedian in a show between 2 comedians, say the word TERMINATE when you finish", llm_config=llm_config, human_input_mode="NEVER", 
                              is_termination_msg= lambda msg: "TERMINATE" in msg["content"])

result = ash_agent.initiate_chat(john_agent, message={"content": "hi, could you tell me a joke? " })

Ash (to John):

hi, could you tell me a joke? 

--------------------------------------------------------------------------------
John (to Ash):

Of course, I'd be happy to share a joke with you. Here it goes:

Why don't scientists trust atoms?

Because they make up everything!

And now, for the punchline I've been saving for this moment, TERMINATE! (Just kidding, I'm not a robot, but I thought it would be funny to include that in my joke.)

--------------------------------------------------------------------------------


In [14]:
agent_with_number = ConversableAgent(
    "agent_with_number",
    system_message="You are playing a game of guess-my-number. "
    "In the first game, you have the "
    "number 53 in your mind, and I will try to guess it. "
    "If I guess too high, say 'too high', if I guess too low, say 'too low'. ",
    llm_config={"config_list": [{"model": "open-mistral-7b", "api_key":os.environ["MISTRAL_API_KEY"] , "api_type": "mistral" }]},
    max_consecutive_auto_reply=1,  # maximum number of consecutive auto-replies before asking for human input
    is_termination_msg=lambda msg: "53" in msg["content"],  # terminate if the number is guessed by the other agent
    human_input_mode="TERMINATE",  # ask for human input until the game is terminated
)

agent_guess_number = ConversableAgent(
    "agent_guess_number",
    system_message="I have a number in my mind, and you will try to guess it. "
    "If I say 'too high', you should guess a lower number. If I say 'too low', "
    "you should guess a higher number. ",
    llm_config={"config_list": [{"model": "open-mistral-7b", "api_key":os.environ["MISTRAL_API_KEY"] , "api_type": "mistral" }]},
    human_input_mode="NEVER",
)

result = agent_with_number.initiate_chat(
    agent_guess_number,
    message="I have a number between 1 and 100. Guess it!",
)

agent_with_number (to agent_guess_number):

I have a number between 1 and 100. Guess it!

--------------------------------------------------------------------------------
agent_guess_number (to agent_with_number):

I'm guessing the number is 50. Is it too high, too low, or just right?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
agent_with_number (to agent_guess_number):

Too low. The number you have in mind is higher than 50.

--------------------------------------------------------------------------------


SDKError: API error occurred: Status 429
{"message":"Requests rate limit exceeded"}